# Data Synthesis or Augmentation
In this script several transformations are applied to either a regular spaced grid (fixed image) or ground truth data (moving images). In that way new imaginary camera projections (moving images) can be created to enlarge the data set.

In [ ]:
import imageio
import numpy as np
import json
from matplotlib import pyplot as plt
from skimage.transform import warp
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from pdb import set_trace

import glob
import math
import nibabel as nib
from endolas import utils
import random
import os

# Functionality
The following functions allow to apply different transformations to x- and y-coordinates of keypoints.
Each function has a parameter that defines some randomness. Following transformations are possible:
- Rotation 
- Translation of all x- and y-coordinates (global)
- Individual translation of each x- and y-coordinate (local)
- Scaling
- Shearing
- Non-linear deformation by applying a sinsusoidal displacement
- Dropout by removing a keypoint from the data

In [ ]:
def random_decision(probability):
    """ Given a probability randomly a true or false decision is made."""
    return random.random() < probability

def apply_sinus(x, y, rand_1, rand_2):
    """ A sinusoidal displacement is applied in x- and y-direction with random amplitude. Spacing 
        and offset is given on a regulary defined grid."""
    y_ratio = (y - offset_y) / (spacing_y * (n_y - 1))
    y_sin = math.sin(y_ratio * math.pi) * y_sin_max * rand_1
    u = x - round(y_sin)
    
    x_ratio = (x - offset_x) / (spacing_x * (n_x - 1))
    x_sin = math.sin(x_ratio * math.pi) * x_sin_max * rand_2
    v = y - round(x_sin)
    
    return u,v

def apply_rot(x, y, rand_1):
    """ Rotation with a limited random angle is applied."""
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    alpha = rot_max * rand_1
    
    vec = np.array([x, y])
    mat = np.array([[np.cos(alpha), -np.sin(alpha)],
                    [np.sin(alpha), np.cos(alpha)]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_scale(x, y, rand_1, rand_2):
    """ Scaling is carried out with limited random magnitude in x- and y-direction."""
    rand_1 = (rand_1 + 1.0) / 2.0
    rand_2 = (rand_1 + 1.0) / 2.0
    
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    x_scale = (1 / x_scale_max) + (x_scale_max - (1 / x_scale_max)) * rand_1
    y_scale = (1 / y_scale_max) + (y_scale_max - (1 / y_scale_max)) * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[x_scale, 0      ],
                    [0      , y_scale]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_shear(x, y, rand_1, rand_2):
    """ Shearing is carried out with limited random magnitude in x- and y-direction."""
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    phi = x_shear_max * rand_1
    psi = y_shear_max * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[1          , np.tan(phi)],
                    [np.tan(psi), 1          ]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_global_trans(x, y, rand_1, rand_2):
    """ In the global translation all keypoints are randomly shifted in x- and y-direction"""
    u = x + round(x_global_max * rand_1)
    v = y + round(y_global_max * rand_2)
    
    return u,v

def apply_local_trans(x, y):
    """ In the local translation each keypoint is randomly shifted in x- and y-direction"""
    u = x + round(x_local_max * (random.random() * 2.0 - 1.0))
    v = y + round(y_local_max * (random.random() * 2.0 - 1.0))
    
    return u,v

def apply_dropout(rand):
    """ A keypoint is randomly removed"""
    dropout = dropout_max * rand
    if random_decision(dropout):
        return True
        
    else:
        return False

def get_transformed_data(rand_1, rand_2, sample_index, index_2_xy):
    """ Given the x- and y-coordinates random transforamtions are applied and the
        transformed coordinates are returned."""
    is_dropout = random_decision(p_dropout)
    is_sinus = random_decision(p_sinus)
    is_rot = random_decision(p_rot)
    is_scale = random_decision(p_scale)
    is_shear = random_decision(p_shear)
    is_global_trans = random_decision(p_global_trans)
    is_local_trans = random_decision(p_local_trans)
    
    moving = np.zeros((height, width))
    index_2_uv = dict()
    
    rand = random.random()
    
    for index, xy in index_2_xy.items():
        x=xy[0]
        y=xy[1]
            
        drop = apply_dropout(rand) if is_dropout else False
          
        if not drop:
            u = x
            v = y

            if is_sinus: u,v = apply_sinus(u, v, rand_1, rand_2)
            if is_rot: u,v = apply_rot(u, v, rand_1) 
            if is_scale: u,v = apply_scale(u, v, rand_1, rand_2) 
            if is_shear: u,v = apply_shear(u, v, rand_1, rand_2) 
            if is_global_trans: u,v = apply_global_trans(u, v, rand_1, rand_2) 
            if is_local_trans: u,v = apply_local_trans(u, v) 

            if u < 0 or v < 0 or u > (width-1) or v > (height-1):
                print('Additional attempt for sample index "{}"'.format(sample_index))
                return None, None, False

            else:
                moving[v][u] = 1
                index_2_uv[index] = (u, v)
            
    return moving, index_2_uv, True

# General Parameters
Following parameters are used to define input and output paths and the shape of the image space.

In [ ]:
# Flag to decide if synthesis or augmentation to carry out
is_synthesis = False

# Define a fixed image if synthesis is carryied out
if is_synthesis:
    fixed_img = 'Data/LASTEN/fix_768_20'

# Directory where to get ground truth data from (if augmentation) and where to store synthesis or augmentation.
base_dir = 'Data/LASTEN/train'

# Parameters for the image space
width = 768
height = 768

n_x = 18
n_y = 18

spacing_x = 32
spacing_y = 32

offset_x = (width - ((n_x - 1) * spacing_x)) / 2.0
offset_y = (height - ((n_y - 1) * spacing_y)) / 2.0

# Define the sample size (amount of augmentations per image)
sample_size = 19

# Amount of smoothing
sigma = 2

# Amount of smoothing for the background
sigma_back= 15

# Transformation Parameters
Following parameters define how often a transformation is applied.

In [ ]:
# Probability that the transformation is applied
p_dropout = 0.7
p_sinus = 0.7
p_rot = 0.7
p_scale = 0.7
p_shear = 0.7
p_global_trans = 0.7
p_local_trans = 0.7

# Limits for the magnitude of each transformation
x_sin_max = 0.15 * height
y_sin_max = 0.015 * width

rot_max = math.pi * 0.1125 # 22.5 deg

x_scale_max = 1.25
y_scale_max = 1.25

x_shear_max = math.pi * 0.05625 # 11.25 deg
y_shear_max = math.pi * 0.05625 # 11.25 deg

x_global_max = 0.2 * width
y_global_max = 0.2 * height

x_local_max = 0.005 * width
y_local_max = 0.005 * height

dropout_max = 0.2

# Synthesis or Augmentation
Carry out the synthesis or augmentation based on fixed or ground truth images.

In [ ]:
# Decide if synthesis or augmentation
if is_synthesis:
    image_ids = [fixed_img]

else:
    globs = glob.glob(base_dir + os.sep + "*.json")
    globs = [int(path.split(os.sep)[-1].split(".")[0]) for path in globs]
    image_ids = sorted(globs)

# The created data will start with this index
global_index = 10000
for image_id in image_ids:
    if is_synthesis:
        sample_path = image_id + ".json"
        
    else:
        sample_path = base_dir + os.sep + "{}.json".format(image_id)
    
    with open(sample_path) as sample_file:
        sample_json_o = json.load(sample_file)
        sample_json = dict()

        for key, value in sample_json_o.items():
            sample_json[int(key)] = value

    for _ in range(0,sample_size):
        index_2_xy = sample_json

        is_valid = False
        while(not is_valid):
            rand_1 = random.random() * 2.0 - 1.0
            rand_2 = random.random() * 2.0 - 1.0

            moving, index_2_uv, is_valid = get_transformed_data(rand_1, rand_2, global_index, index_2_xy)

        json_file = json.dumps(index_2_uv)
        f = open(base_dir + os.sep + "{}.json".format(global_index), "w")
        f.write(json_file)
        f.close()

        moving = utils.apply_smoothing(moving, sigma=sigma, sigma_back=sigma_back)
        plt.imsave(base_dir + os.sep + "{}_mov.png".format(global_index), moving, cmap="gray")

        global_index += 1